# LP goat

In [28]:
import random
import time
import numpy as np
import gurobipy as gp
from gurobipy import GRB
from scipy.optimize import linprog  # HiGHS solver through SciPy interface

# Problem size
n_vars = 30000
n_constraints = 3000
random.seed(42)

# Shared data - generating a standard LP problem with denser constraints
var_costs = [random.uniform(1, 10) for _ in range(n_vars)]

# Create denser constraints (more variables per constraint)
constraint_data = []
for _ in range(n_constraints):
    # Include more variables in each constraint (increased from 15 to 50)
    vars_in = random.sample(range(n_vars), 1000)  
    coeffs = [random.uniform(0.5, 3.0) for _ in range(len(vars_in))]
    
    # To ensure feasibility, we make RHS a fraction of possible sum
    # This guarantees feasibility when variables are set to reasonable values
    rhs = sum(coeffs) * 0.5  # Relaxed to ensure feasibility
    constraint_data.append((vars_in, coeffs, rhs))

print(f"Solving LP problem with {n_vars} variables and {n_constraints} constraints")
print(f"Each constraint contains 100 variables (denser formulation)")

# ------------------ GUROBI ------------------
model_gurobi = gp.Model("compare_lp")
model_gurobi.setParam("OutputFlag", 0)  # Suppress output

# Add continuous variables bounded between 0 and unlimited
xg = model_gurobi.addVars(n_vars, lb=0, ub=GRB.INFINITY, name="xg")

# Set objective: minimize cost
model_gurobi.setObjective(gp.quicksum(var_costs[i] * xg[i] for i in range(n_vars)), GRB.MINIMIZE)

# Add constraints
for vars_in, coeffs, rhs in constraint_data:
    model_gurobi.addConstr(gp.quicksum(coeffs[i] * xg[vars_in[i]] for i in range(len(vars_in))) >= rhs)

start_gurobi = time.time()
model_gurobi.optimize()
end_gurobi = time.time()

# ------------------ Results ------------------
print("\n====== Gurobi ======")
if model_gurobi.status == GRB.OPTIMAL:
    print(f"✅ Gurobi cost: {model_gurobi.objVal:.6f}")
    print(f"⏱️ Time: {end_gurobi - start_gurobi:.6f} sec")
    print(f"⚙️ Constraints: {model_gurobi.NumConstrs}")
    print(f"⚙️ Non-zeros in constraint matrix: {model_gurobi.NumNZs}")
else:
    print("❌ Gurobi did not solve optimally. Status:", model_gurobi.status)

# You can uncomment the HiGHS portion if you want to run that comparison

Solving LP problem with 30000 variables and 3000 constraints
Each constraint contains 100 variables (denser formulation)

====== Gurobi ======
✅ Gurobi cost: 19698.680015
⏱️ Time: 3.260233 sec
⚙️ Constraints: 3000
⚙️ Non-zeros in constraint matrix: 3000000


# MILP

In [9]:
import random
import time
import gurobipy as gp
from gurobipy import GRB
import numpy as np
from scipy.optimize import milp  # HiGHS solver through SciPy interface

# Problem size
n_vars = 400
n_constraints = 160
random.seed(42)

# Shared data
var_weights = [(i % 10 + 1) for i in range(n_vars)]
constraint_data = []
for _ in range(n_constraints):
    vars_in = random.sample(range(n_vars), 10)
    coeffs = [random.randint(1, 3) for _ in range(len(vars_in))]
    rhs = sum(coeffs) // 2  # ensures feasibility
    constraint_data.append((vars_in, coeffs, rhs))

# ------------------ GUROBI ------------------
model_gurobi = gp.Model("compare_milp")
model_gurobi.setParam("OutputFlag", 0)
xg = model_gurobi.addVars(n_vars, vtype=GRB.BINARY, name="xg")
model_gurobi.setObjective(gp.quicksum(var_weights[i] * xg[i] for i in range(n_vars)), GRB.MINIMIZE)

for vars_in, coeffs, rhs in constraint_data:
    model_gurobi.addConstr(gp.quicksum(coeffs[i] * xg[vars_in[i]] for i in range(10)) >= rhs)

start_gurobi = time.time()
model_gurobi.optimize()
end_gurobi = time.time()

# # ------------------ HiGHS ------------------
# # Set up the objective coefficients
# c = np.zeros(n_vars)
# for i in range(n_vars):
#     c[i] = var_weights[i]

# # Set up the constraints in SciPy's format
# A_ub = np.zeros((n_constraints, n_vars))
# b_ub = np.zeros(n_constraints)

# for i, (vars_in, coeffs, rhs) in enumerate(constraint_data):
#     # For HiGHS, we need to convert >= constraints to <= constraints
#     # So we multiply by -1
#     for j in range(len(vars_in)):
#         A_ub[i, vars_in[j]] = -coeffs[j]
#     b_ub[i] = -rhs

# # Set variable bounds (binary variables: either 0 or 1)
# bounds = [(0, 1) for _ in range(n_vars)]
# integrality = np.ones(n_vars)  # 1 indicates integer variable

# start_highs = time.time()
# result = milp(
#     c=c,
#     constraints=[
#         {"A": A_ub, "b": b_ub, "type": "<="}
#     ],
#     bounds=bounds,
#     integrality=integrality,
#     options={"solver": "highs"}
# )
# end_highs = time.time()

# ------------------ Results ------------------
print("====== Gurobi ======")
if model_gurobi.status == GRB.OPTIMAL:
    print(f"✅ Gurobi cost: {model_gurobi.objVal:.2f}")
    print(f"⏱️ Time: {end_gurobi - start_gurobi:.4f} sec")
else:
    print("❌ Gurobi did not solve optimally.")

# print("\n====== HiGHS ======")
# if result.success:
#     print(f"✅ HiGHS cost: {result.fun:.2f}")
#     print(f"⏱️ Time: {end_highs - start_highs:.4f} sec")
# else:
#     print(f"❌ HiGHS did not solve optimally. Status: {result.status}")
#     print(f"Message: {result.message}")

====== Gurobi ======
✅ Gurobi cost: 687.00
⏱️ Time: 8.6186 sec


# LP comparison

In [17]:
import random
import time
import numpy as np
import gurobipy as gp
from gurobipy import GRB
from scipy.optimize import linprog  # HiGHS solver through SciPy interface

# Problem size
n_vars = 3000
n_constraints = 2000
random.seed(42)

# Shared data - generating a standard LP problem
var_costs = [random.uniform(1, 10) for _ in range(n_vars)]
constraint_data = []
for _ in range(n_constraints):
    vars_in = random.sample(range(n_vars), 15)
    coeffs = [random.uniform(0.5, 3.0) for _ in range(len(vars_in))]
    rhs = sum(coeffs) * 0.7  # This ensures constraints are not too tight
    constraint_data.append((vars_in, coeffs, rhs))

print(f"Solving LP problem with {n_vars} variables and {n_constraints} constraints")

Solving LP problem with 3000 variables and 2000 constraints


In [18]:


# ------------------ GUROBI ------------------
model_gurobi = gp.Model("compare_lp")
model_gurobi.setParam("OutputFlag", 0)  # Suppress output

# Add continuous variables bounded between 0 and 1
xg = model_gurobi.addVars(n_vars, lb=0.0, ub=1.0, name="xg")

# Set objective: minimize cost
model_gurobi.setObjective(gp.quicksum(var_costs[i] * xg[i] for i in range(n_vars)), GRB.MINIMIZE)

# Add constraints
for vars_in, coeffs, rhs in constraint_data:
    model_gurobi.addConstr(gp.quicksum(coeffs[i] * xg[vars_in[i]] for i in range(len(vars_in))) >= rhs)

start_gurobi = time.time()
model_gurobi.optimize()
end_gurobi = time.time()

# # ------------------ HiGHS ------------------
# # Set up the objective coefficients
# c = np.array(var_costs)

# # Set up the constraints in SciPy's format (Ax >= b => -Ax <= -b for linprog)
# A_ub = np.zeros((n_constraints, n_vars))
# b_ub = np.zeros(n_constraints)

# for i, (vars_in, coeffs, rhs) in enumerate(constraint_data):
#     # Convert from >= constraint to <= constraint by multiplying by -1
#     for j in range(len(vars_in)):
#         A_ub[i, vars_in[j]] = -coeffs[j]
#     b_ub[i] = -rhs

# # Set variable bounds (continuous variables from 0 to 1)
# bounds = [(0, 1) for _ in range(n_vars)]

# start_highs = time.time()
# result = linprog(
#     c=c,
#     A_ub=A_ub,
#     b_ub=b_ub,
#     bounds=bounds,
#     method='highs'  # Explicitly specify HiGHS solver
# )
# end_highs = time.time()

# ------------------ Results ------------------
print("\n====== Gurobi ======")
if model_gurobi.status == GRB.OPTIMAL:
    print(f"✅ Gurobi cost: {model_gurobi.objVal:.6f}")
    print(f"⏱️ Time: {end_gurobi - start_gurobi:.6f} sec")
else:
    print("❌ Gurobi did not solve optimally.")
    
# print("\n====== HiGHS ======")
# if result.success:
#     print(f"✅ HiGHS cost: {result.fun:.6f}")
#     print(f"⏱️ Time: {end_highs - start_highs:.6f} sec")
#     print(f"⚙️ Iterations: {result.nit}")
# else:
    # print(f"❌ HiGHS did not solve optimally. Status: {result.status}")
    # print(f"Message: {result.message}")

# # Compare solution values
# if model_gurobi.status == GRB.OPTIMAL and result.success:
#     print("\n====== Solution Comparison ======")
#     print(f"Solution difference: {abs(model_gurobi.objVal - result.fun):.10f}")
    
#     # Sample a few variable values to compare
#     sample_vars = random.sample(range(n_vars), 5)
#     print("\nSample variable values:")
#     for i in sample_vars:
#         gurobi_val = xg[i].X
#         highs_val = result.x[i]
#         print(f"Var {i}: Gurobi = {gurobi_val:.6f}, HiGHS = {highs_val:.6f}, Diff = {abs(gurobi_val - highs_val):.8f}")


====== Gurobi ======
✅ Gurobi cost: 9505.846390
⏱️ Time: 0.506514 sec
